In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import requests
import time

import pdfkit
import pyautogui
import playwright

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from bs4 import BeautifulSoup

You have to have access for sciencedirect. Before you start to run the script, you have to login to sciencedirect.

https://www.sciencedirect.com/

In [30]:
DEMO = True
MAX_PAPERS_TO_DOWNLOAD = 3
# you have to have access for sciencedirect. Before you start to run the script, you have to login to sciencedirect.
# https://www.sciencedirect.com/

#### Preparation

In [31]:
# Function to extract the base domain of a URL (until .com, .org, etc.)
def extract_base_domain(url):
    if pd.notna(url):
        # Split the URL and return the main domain (e.g., 'arxiv.org', 'ieee.org')
        return '.'.join(url.split('/')[2].split('.')[-2:])
    return None
file_path = '../Scrapes_ALL.csv'
papers_df = pd.read_csv(file_path)
display(papers_df.head(3))
# Apply the function to extract base domains
papers_df['Base_Domain'] = papers_df['URL'].apply(extract_base_domain)


,Unnamed: 0,Title,Authors,Year,Cited By,Detected_Dataset,Detected_Topic,Abstract,DOI,Journal,...,Mentions_Statistical_significance,Mentions_P-value,Mentions_T-test,Mentions_Anova,Mentions_Correlation,Mentions_Regression,Mentions_Baseline_comparison,Mentions_Mae,Mentions_Rmse,Mentions_Bias
0,0,10 Automated Face Analysis for Affective Compu...,"['JF Cohn', 'F De la Torre']",2015,170,Affective Faces Database,classifier,Differences in manual coding between databases...,No DOI,The Oxford handbook of affective …,...,False,False,False,False,False,False,False,False,False,False
1,1,3D facial expression recognition based on auto...,"['H Tang', 'TS Huang']",2008,205,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",facial expression recognition from 3D facial s...,No DOI,… on computer vision and pattern recognition …,...,False,False,False,False,False,False,False,False,False,False
2,2,3D facial expression recognition based on prim...,"['J Wang', 'L Yin', 'X Wei', 'Y Sun']",2006,440,Binghamton University 3D Facial Expression,facial expression recognition,expressions using 3D facial expression range d...,No DOI,… Vision and Pattern Recognition …,...,False,False,False,False,False,False,False,False,False,False


# Get papers from scienecedirect.com 

Init

In [32]:
scienece_direct_df = papers_df[papers_df['URL'].str.contains('sciencedirect.com', case=False, na=False)]
scienece_direct_df = scienece_direct_df.rename(columns={'Unnamed: 0': 'ID'})
# shuffle for fun
scienece_direct_df = scienece_direct_df.sample(frac=1).reset_index(drop=True)
scienece_direct_df = scienece_direct_df.reset_index(drop=True)
len(scienece_direct_df['URL'].to_list())
print(f"Number of papers: {len(scienece_direct_df['URL'].to_list())}")

Number of papers: 40


### Main Script

After starting the script, it will open the Firefox web browser. You will need to enter your ScienceDirect credentials there. Do not forget to push the "Log in" button.

Please be patient, as it may take some time for the script to start saving the papers based on the links from the `science_direct_df` dataset. The script includes several sleep statements of about 30 seconds to ensure proper loading and execution.

The outcome of the script will be `html` files saved in the `x_research_papers_save\1_raw_files` folder. Each html file is one paper. These html files will be processed in the next script.

In [33]:

# make sure that folder is exist
DOWNLOAD_DIR = os.path.join(os.path.dirname(os.path.abspath('.')), "x_research_papers_save", "1_raw_files")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

print(DOWNLOAD_DIR)

def login_to_sciencedirect(driver, username="", password=""):
    login_url = "https://id.elsevier.com/as/authorization.oauth2?platSite=SD%2Fscience&additionalPlatSites=GH%2Fgeneralhospital%2CMDY%2Fmendeley%2CSC%2Fscopus&scope=openid%20email%20profile%20els_auth_info%20els_idp_info%20els_idp_analytics_attrs%20els_sa_discover%20urn%3Acom%3Aelsevier%3Aidp%3Apolicy%3Aproduct%3Aindv_identity&response_type=code&redirect_uri=https%3A%2F%2Fwww.sciencedirect.com%2Fuser%2Fidentity%2Flanding&authType=SINGLE_SIGN_IN&prompt=login&client_id=SDFE-v4&state=retryCounter%3D0%26csrfToken%3D0007d31c-b511-4d2f-962c-46977ec5d3c7%26idpPolicy%3Durn%253Acom%253Aelsevier%253Aidp%253Apolicy%253Aproduct%253Aindv_identity%26returnUrl%3D%252F%26prompt%3Dlogin&els_policy=idp_policy_indv_identity_plus"
    driver.get(login_url)
    
    try:
        # Wait for and click the "Register with institution" button
        register_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "bdd-elsPrimaryBtn"))
        )
        register_button.click()
        print("Clicked 'Register with institution' button")
    except TimeoutException:
        print("'Register with institution' button not found or not clickable")
        return False
    
    # Wait for the page to load
    time.sleep(30)
    
    try:
        # Now we're on the institutional login page
        # Enter username and password
        username_field = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "username"))
        )
        username_field.clear()  # Clear any pre-filled text
        username_field.send_keys(username)
        print("Entered username")
        
        password_field = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "password"))
        )
        password_field.clear()  # Clear any pre-filled text
        password_field.send_keys(password)
        print("Entered password")
        
        # Click the login button
        login_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.NAME, "action"))
        )
        login_button.click()
        print("Clicked login button")
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Error during login process: {str(e)}")
        return False
    
    # Wait for login to complete and redirect
    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.ID, "sdMainContent"))
        )
        print("Login successful - redirected to ScienceDirect main page")
        return True
    except TimeoutException:
        print("Login may have failed or page did not load completely")
        return False


    driver.get(url)
    print("We are in download pdf function")
    
    try:
        # Wait for the content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".article-wrapper, #sdMainContent, .Publication"))
        )
        print("Content loaded")

        # Scroll to ensure all content is loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for any dynamic content to load after scrolling

        # Get the HTML content
        html_content = driver.execute_script("return document.documentElement.outerHTML")
        print("HTML content captured. Length:", len(html_content))
        
        # Save as HTML
        html_filename = f"{filename}.html"
        html_path = os.path.join(DOWNLOAD_DIR, html_filename)
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
            print(f"Saved loaded page as HTML: {html_filename}")

        # Save as PDF
        pdf_filename = f"{filename}.pdf"
        pdf_path = os.path.join(DOWNLOAD_DIR, pdf_filename)
        try:
            pdfkit.from_string(html_content, pdf_path)
            print(f"Saved loaded page as PDF: {pdf_filename}")
        except Exception as e:
            print(f"Error saving loaded page as PDF: {str(e)}")

        return True

    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return False


def save_content(driver, url, filename):
    driver.get(url)
    print(f"Processing: {url}")
    
    try:
        # Wait for the content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".article-wrapper, #sdMainContent, .Publication"))
        )
        print("Content loaded")

        # Scroll to ensure all content is loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for any dynamic content to load after scrolling

        # Get the HTML content
        html_content = driver.execute_script("return document.documentElement.outerHTML")
        print("HTML content captured. Length:", len(html_content))
        # print(html_content)
        
        # Save as HTML
        html_filename = f"{filename}.html"
        html_path = os.path.join(DOWNLOAD_DIR, html_filename)
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"Saved loaded page as HTML: {html_filename}")

        # multiple method to save pdf, onyl one was working. Sad.
        pdf_filename = f"{filename}.pdf"
        pdf_path = os.path.join(DOWNLOAD_DIR, pdf_filename)

        # Method 1: Use browser's built-in PDF printing (Chrome)
        """
        try:
            print("Attempting Method 1: Browser's built-in PDF printing")
            driver.execute_script('window.print();')
            time.sleep(5)  # Wait for the print dialog to process
            pyautogui.hotkey('ctrl', 'shift', 'p')  # Change printer to "Save as PDF"
            time.sleep(1)
            pyautogui.press('enter')  # Select "Save as PDF"
            time.sleep(1)
            pyautogui.write(pdf_path)  # Type the file path
            time.sleep(1)
            pyautogui.press('enter')  # Save the PDF

            print(f"PDF should be saved as: {pdf_path}")
            time.sleep(5)  # Wait for the save process to complete
            if os.path.exists(pdf_path):
                print(f"PDF successfully saved: {pdf_filename}")
            
            else:
                print(f"PDF was not saved at the expected location: {pdf_path}")
            print("Method 1 completed. Check your downloads folder for the PDF.")
        except Exception as e:
            print(f"Method 1 failed: {str(e)}")

        # Method 2: Use pdfkit (if installed)
        try:
            print("Attempting Method 2: pdfkit")
            import pdfkit
            pdfkit.from_string(html_content, pdf_path)
            print(f"Method 2 succeeded. Saved PDF: {pdf_filename}")
            return True
        except Exception as e:
            print(f"Method 2 failed: {str(e)}")

        # Method 3: Use WeasyPrint (if installed)
        try:
            print("Attempting Method 3: WeasyPrint")
            from weasyprint import HTML
            HTML(string=html_content).write_pdf(pdf_path)
            print(f"Method 3 succeeded. Saved PDF: {pdf_filename}")
            return True
        except Exception as e:
            print(f"Method 3 failed: {str(e)}")

        # Method 4: Use Playwright to generate PDF (if installed)
        try:
            print("Attempting Method 4: Playwright")
            from playwright.sync_api import sync_playwright
            with sync_playwright() as p:
                browser = p.chromium.launch()
                page = browser.new_page()
                page.set_content(html_content)
                page.pdf(path=pdf_path)
                browser.close()
            print(f"Method 4 succeeded. Saved PDF: {pdf_filename}")
            return True
        except Exception as e:
            print(f"Method 4 failed: {str(e)}")

        
        
        with open(html_path) as f:
            pdfkit.from_file(f, f"{filename}.pdf")
            print(f"Saved loaded page as pdf: {html_filename}")

        # Extract and save text content
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content = soup.get_text(separator='\n', strip=True)
        
        text_filename = f"{filename}.txt"
        text_path = os.path.join(DOWNLOAD_DIR, text_filename)
        with open(text_path, 'w', encoding='utf-8') as f:
            f.write(text_content)
        print(f"Saved text content: {text_filename}")
        """
        return True
    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return False
   
def main():


    # Initialize the WebDriver (make sure you have the appropriate driver installed)
    driver = webdriver.Chrome()  # or webdriver.Firefox(), etc.
    
    try:
        # Login to ScienceDirect
        login_to_sciencedirect(driver)
        
        # Iterate through the URLs and download PDFs
        c = 0
        for index, row in scienece_direct_df.iterrows():
            if DEMO and c >= MAX_PAPERS_TO_DOWNLOAD:
                break
            else:
                ID = row["ID"]
                print(f" Processing paper {ID}")
                url = row['URL']
                # Generate a filename based on some criteria, e.g., the paper's title
                filename = f"science_direct_{ID}"
                # success = download_pdf(driver, url, filename)
                success = save_content(driver, url, filename)
                if success:
                    scienece_direct_df.at[index, 'Downloaded'] = 'Yes'
                else:
                    scienece_direct_df.at[index, 'Downloaded'] = 'No'

            c += 1
        
        # Save the updated DataFrame
        scienece_direct_df.to_csv('sciencedirect_papers_status.csv', index=False)
    
    finally:
        # Close the browser
        driver.quit()

if __name__ == "__main__":
    main()

e:\ITU-BOOK\Data in Wild\Project\DataInWild\SUBMISSION\1_scraping\x_research_papers_save\1_raw_files
Clicked 'Register with institution' button
Error during login process: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF75EE438A5+3004357]
	(No symbol) [0x00007FF75EAD9970]
	(No symbol) [0x00007FF75E98582A]
	(No symbol) [0x00007FF75E9D5B8E]
	(No symbol) [0x00007FF75E9D5E7C]
	(No symbol) [0x00007FF75EA1EC27]
	(No symbol) [0x00007FF75E9FBC1F]
	(No symbol) [0x00007FF75EA1BA4C]
	(No symbol) [0x00007FF75E9FB983]
	(No symbol) [0x00007FF75E9C7628]
	(No symbol) [0x00007FF75E9C8791]
	GetHandleVerifier [0x00007FF75EE6A00D+3161901]
	GetHandleVerifier [0x00007FF75EEBE060+3506048]
	GetHandleVerifier [0x00007FF75EEB400D+3465005]
	GetHandleVerifier [0x00007FF75EC30EEB+830987]
	(No symbol) [0x00007FF75EAE467F]
	(No symbol) [0x00007FF75EAE09D4]
	(No symbol) [0x00007FF75EAE0B6D]
	(No symbol) [0x00007FF75EAD0149]
	BaseThreadInitThunk [0x00007FFAF182E8D7+23]
	RtlUserThreadStart [0x00007FFAF303FBCC+44]

 